# Welche Programmierverstöße gibt es?

## 1. Verbindung zur Datenbank
Es wird eine Verbindung zur Neo4j-Datenbank aufgebaut.

In [8]:
import py2neo

graph = py2neo.Graph(bolt=True, host='localhost', user='neo4j', password='neo4j')

## 2. Cypher-Abfrage
Es wird eine Abfrage an die Datenbank gestellt. Das Ergebnis wird in einem Dataframe (pandas) gespeichert.

In [9]:
import pandas as pd

query ="MATCH (:Report)-[:HAS_FILE]->(file:File:Pmd)-[:HAS_VIOLATION]->(violation:Violation) RETURN  violation.ruleSet as category, count(violation.ruleSet) as frequency"
df = pd.DataFrame(graph.run(query).data())


## 3. Datenaufbereitung
Zur Kontrolle werden die ersten fünf Zeilen des Ergebnisses der Abfrage als Tabelle ausgegeben.

In [10]:
df.head()

,category,frequency
0,Code Style,27
1,Multithreading,1
2,Best Practices,4
3,Design,1
4,Error Prone,32


## 4. Visualisierung
Die Daten werden mittels eines Radar Charts visualisiert (https://plot.ly/python/radar-chart/, r=frequency, theta=category).

In [11]:

frequency=df['frequency'].tolist()


category=df['category'].tolist()

import plotly.plotly as py
import plotly.graph_objs as go

data = [go.Scatterpolar(
  r = frequency,
  theta = category,
  fill = 'toself'
)]

layout = go.Layout(
  polar = dict(
    radialaxis = dict(
      visible = True,
      range = [0, 50]
    )
  ),
  showlegend = False
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename = "radar")

In [13]:
from IPython.display import display, HTML

base_html = """
<!DOCTYPE html>
<html>
  <head>
  <script type="text/javascript" src="http://kozea.github.com/pygal.js/javascripts/svg.jquery.js"></script>
  <script type="text/javascript" src="https://kozea.github.io/pygal.js/2.0.x/pygal-tooltips.min.js""></script>
  </head>
  <body>
    <figure>
      {rendered_chart}
    </figure>
  </body>
</html>
"""

In [16]:
import pygal
radar_chart = pygal.Radar()
radar_chart.title = 'Programmierverstöße'
radar_chart.x_labels = category
radar_chart.add('Anzahl der Programmierverstöße', frequency)

display(HTML(base_html.format(rendered_chart=radar_chart.render(is_unicode=True))))

In [ ]:
for einzelneKategorie in category:
    index= category.index(einzelneKategorie)
    #dazugehörige frequency herausbekommen
    häufigkeit=frequency[index]
    radar_chart.add(einzelneKategorie,häufigkeit)